# Send REST requests to the API running in Aspire

In [ ]:
var baseUrl = "https://localhost:7220";

## Healthcheck 💉

In [8]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/

Name,Value
traceparent,00-170bb7c928f14e3e21a6252cbadd58c2-9f66f98b29819626-00
Name,Value
Date,"Fri, 18 Apr 2025 22:00:59 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,text/plain; charset=utf-8
Content-Length,26


## GET Weather Forecast 🌡️☁️

In [9]:
#!set --value @csharp:baseUrl --name baseUrl
GET {{baseUrl}}/v1/weather-forecast/Bristol/{{$datetime iso8601}}

Name,Value
traceparent,00-bbbea896bc84e18e9640173416717226-5c619e6e1939c90c-00
Name,Value
Date,"Fri, 18 Apr 2025 22:01:04 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,175
requestedRegion,"""Bristol"""
requestedDate,"""2025-04-18T22:01:05.1032594Z"""
requestId,"""da3e7010-6afd-46ec-bacb-dccab2f2a63f"""


## POST Colected Weather Data 🌦️🌈🔆

In [25]:
string[] fruits = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "watermelon", "guava"];
var randomReference = $"{fruits[new Random().Next(fruits.Length)]}-{Random.Shared.Next(10, 99)}";
randomReference.Display();

fig-55

In [26]:
#!set --value @csharp:baseUrl --name baseUrl
#!set --value @csharp:randomReference --name randomReference

POST {{baseUrl}}/v1/collected-weather-data/Bristol/{{randomReference}}
Content-Type: application/json
    
{
    "points": [
        {
            "time": "{{$datetime iso8601}}",
            "windSpeedInMetersPerSecond": {{$randomInt 0 69}}.{{$randomInt 0 99}},
            "windDirection": "N",
            "temperatureReadingInDegreesCelcius": {{$randomInt -15 40}}.{{$randomInt 0 99}},
            "humidityReadingInPercent": {{$randomInt 20 100}}
        }
    ]
}

Name,Value
traceparent,00-1ea691ae1dabf72b031821e5227c77d7-87a398eddd7f635b-00
Content-Type,application/json
Content-Length,291
Name,Value
Date,"Fri, 18 Apr 2025 22:07:10 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,73
requestId,"""04ddc24d-02b0-4768-ad66-ce5fdfd626ce"""


## GET Traces 🤖

In [59]:
using System.Net.Http;
using System.Text.Json;

public class TraceData
{
    public string Resource { get; set; } = string.Empty;
    public string Source { get; set; } = string.Empty;
    public string DisplayName { get; set; } = string.Empty;
    public string TraceId { get; set; } = string.Empty;
    public string SpanId { get; set; } = string.Empty;    
    public Dictionary<string, object?> Tags { get; set; } = new();

    public override string ToString()
    {
        var thirdPart = string.Empty;
        if(DisplayName == "Domain Event Insertion")
            thirdPart = Tags["domain-event.eventclassName"].ToString().Split('.').LastOrDefault();

        if(DisplayName == "User Notication Sent")
            thirdPart = $"🎉✉️ reference: {Tags["user-notification-event.reference"].ToString()}";

        if(DisplayName == "Outbox Item Insertion")
            thirdPart = $"📫 outbox id: {Tags["outbox-item.Id"].ToString()}";
        
            
        {
            return $"{Resource} | {DisplayName} | {thirdPart}";
        }
        return $"{Resource} | {DisplayName} | {Tags["domain-event.eventclassName"]?? string.Empty}";
    } 
        
}

var client = new HttpClient() { BaseAddress = new Uri("https://localhost:7059") };
var response = client.GetAsync("traces").GetAwaiter().GetResult();
var responseJson = response.Content.ReadAsStringAsync().GetAwaiter().GetResult();
JsonSerializer.Deserialize<TraceData[]>(responseJson, JsonSerializerOptions.Web ).Display();

index value 0 WeatherApp.EventListener | Domain Event Insertion | ModelUpdated Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 4930a620bb38bbd8 Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "8" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelUpdated" 1 NotificationService | User Notication Sent | 🎉✉️ reference: fig-55 Resource NotificationService Source UserNotificationEventHandler DisplayName User Notication Sent TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 9ac5ea7456ba8aa6 Tags key value user-notification-event.body "Dear user, your data has been submitted and included in our latest model" user-notification-event.reference "fig-55" user-notification-event.timestamp "2025-04-18T22:07:13.9870625+00:00" 2 WeatherApp.EventListener | Outbox Item Insertion | 📫 outbox id: 10060 Resource WeatherApp.EventListener Source OutboxRepository DisplayName Outbox Item Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 9b07806af6819cab Tags key value outbox-item.TypeName "UserNotificationEvent" outbox-item.Id 10060 3 WeatherApp.EventListener | Domain Event Insertion | ModelingDataAccepted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId c9ce011e2ad6d15b Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "5" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.ModelingDataAccepted" 4 WeatherApp.API | Domain Event Insertion | LocationIdFound Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId dd72fde665383e4d Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "2" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.LocationIdFound" 5 WeatherApp.EventListener | Domain Event Insertion | SubmissionComplete Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 7be163a8ae1b0a3f Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "7" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmissionComplete" 6 WeatherApp.EventListener | Domain Event Insertion | PendingContributorPaymentCommitted Resource WeatherApp.EventListener Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 7b6f20db30af1332 Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "6" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentCommitted" 7 WeatherApp.API | Domain Event Insertion | SubmittedToModeling Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId a168c306f1e42bf3 Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "4" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.SubmittedToModeling" 8 WeatherApp.API | Domain Event Insertion | PendingContributorPaymentPosted Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 05d1b58f8e7ed917 Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "3" domain-event.eventclassName "WeatherApp.Domain.DomainEvents.PendingContributorPaymentPosted" 9 WeatherApp.API | Domain Event Insertion | WeatherDataCollectionInitiated Resource WeatherApp.API Source EventRepositorySql DisplayName Domain Event Insertion TraceId 1ea691ae1dabf72b031821e5227c77d7 SpanId 00fce3c1ba711d21 Tags key value domain-event.streamId "04ddc24d-02b0-4768-ad66-ce5fdfd626ce" domain-event.version "1" doma